In [ ]:
import Pkg
Pkg.activate(".")

# not for the faint of heart!
# Pkg.update()

pkgs = [
"ArgParse",
"Base64",
"BioSequences",
"DataFrames",
"Dates",
"DelimitedFiles",
"FASTX",
"Statistics",
"StatsPlots",
"uCSV",
"Revise",
"StatsBase"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

import Mycelia

In [ ]:
data_dir = joinpath(dirname(pwd()), "data")

In [ ]:
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
SRR_paths = filter(x -> isfile(joinpath(x, "megahit", "final.contigs.fastg.gfa.fna")), SRR_paths)
# filter out samples that have already been processed
# SRR_paths = filter(x -> !isfile(joinpath(x, basename(x) * ".contig_info.tsv")), SRR_paths)

In [ ]:
# mamba install -c conda-forge -c bioconda virsorter=2

# mamba create -n vs2 -c conda-forge -c bioconda virsorter=2
# mamba activate vs2

In [ ]:
virsorter_db_dir = mkpath("$(homedir())/workspace/virsorter2")

# got conda errors with this

# (base) 2023-06-06T16:05:47 jovyan@76eefed4ebe2:~/workspace/Mycelia
# $ virsorter setup -d /home/jovyan/workspace/virsorter2 -j 4
# Traceback (most recent call last):
#   File "/opt/conda/bin/virsorter", line 6, in <module>
#     from virsorter.virsorter import cli
#   File "/opt/conda/lib/python3.9/site-packages/virsorter/virsorter.py", line 11, in <module>
#     from snakemake import load_configfile
#   File "/opt/conda/lib/python3.9/site-packages/snakemake/__init__.py", line 21, in <module>
#     from snakemake.workflow import Workflow
#   File "/opt/conda/lib/python3.9/site-packages/snakemake/workflow.py", line 23, in <module>
#     from snakemake.rules import Rule, Ruleorder, RuleProxy
#   File "/opt/conda/lib/python3.9/site-packages/snakemake/rules.py", line 54, in <module>
#     from snakemake.common import Mode, lazy_property, TBDInt
# ImportError: cannot import name 'TBDInt' from 'snakemake.common' (/opt/conda/lib/python3.9/site-packages/snakemake/common/__init__.py)

# run(`virsorter setup -d $(virsorter_db_dir) -j 4`)
# run(`https://osf.io/v46sc/download`)

db_tgz_path = joinpath(virsorter_db_dir, "db.tgz")
if !isfile(db_tgz_path)
    download("https://osf.io/v46sc/download", joinpath(virsorter_db_dir, "db.tgz"))
end
db_dir = replace(db_tgz_path, ".tgz" => "")
if !isdir(db_dir)
    run(`tar -xzf $(db_tgz_path) --directory=$(virsorter_db_dir)`)
end

In [ ]:
# mamba create -n vs2 -c conda-forge -c bioconda virsorter=2
# $  mamba activate vs2
# (vs2) 2023-06-06T16:26:44 jovyan@76eefed4ebe2:~
# $ virsorter config --init-source --db-dir=/home/jovyan/workspace/virsorter2/db
# [2023-06-06 16:27 INFO] VirSorter 2.2.4
# [2023-06-06 16:27 INFO] /opt/conda/envs/vs2/bin/virsorter config --init-source --db-dir=/home/jovyan/workspace/virsorter2/db
# [2023-06-06 16:27 INFO] saving /home/jovyan/workspace/virsorter2/db as DBDIR to config file /opt/conda/envs/vs2/lib/python3.10/site-packages/virsorter/template-config.yaml

# # run(`virsorter config --init-source --db-dir=$(db_dir)`)

In [ ]:
Sys.CPU_THREADS

In [ ]:
# # fetch testing data
# wget -O test.fa https://raw.githubusercontent.com/jiarong/VirSorter2/master/test/8seq.fa
# # run classification with 4 threads (-j) and test-out as output diretory (-w)
# virsorter run -w test.out -i test.fa --min-length 1500 -j 4 all
# ls test.out

In [ ]:
SRR_path = SRR_paths[1]
# SRR_path = SRR_paths[2]

In [ ]:
SRR = basename(SRR_path)

In [ ]:
assembled_fasta = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.fna")

In [ ]:
virsorter_dir = mkpath(joinpath(SRR_path, "virsorter"))

In [ ]:
# works in standalone conda environment but not general installation
run(`virsorter run -w $(virsorter_dir) -i $(assembled_fasta) --min-length 1 -j $(Sys.CPU_THREADS) all`)